<a href="https://colab.research.google.com/github/german1728/Chimera/blob/master/Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 3
# Ajuste del modelo: intervalos de confianza y bootstrapping

**Content creators**: Pierre-Étienne Fiquet, Anqi Wu, Alex Hyafil with help from Byron Galbraith

**Content reviewers**: Lina Teichmann, Saeed Salehi, Patrick Mineault, Ella Batty, Michael Waskom 

**Content traduction**: Israel Chaparro

Source: Neuromatch Academy

#Objetivos del tutorial

Este es el Tutorial 3 de una serie sobre cómo ajustar modelos a datos. Comenzamos con regresión lineal simple, usando optimización de mínimos cuadrados (Tutorial 1) y Estimación de máxima verosimilitud (Tutorial 2). Usaremos bootstrapping para construir intervalos de confianza alrededor de los parámetros del modelo lineal inferido (Tutorial 3). Terminaremos nuestra exploración de modelos de regresión generalizando a la regresión lineal múltiple y la regresión polinomial (Tutorial 4). Terminamos aprendiendo a elegir entre estos distintos modelos. Analizamos la compensación de sesgo-varianza (Tutorial 5) y la Validación cruzada para la selección del modelo (Tutorial 6).

En este tutorial, discutiremos cómo medir qué tan buenos son nuestros parámetros estimados del modelo.
- Aprenda a usar bootstrapping para generar nuevos conjuntos de datos de muestra
- Estime el parámetro de nuestro modelo en estos nuevos conjuntos de datos de muestra.
- Cuantificar la varianza de nuestra estimación utilizando intervalos de confianza.

Hasta este punto, hemos estado encontrando formas de estimar los parámetros del modelo para ajustar algunos datos observados. Nuestro enfoque ha sido optimizar algún criterio, ya sea minimizar el error cuadrático medio o maximizar la probabilidad al usar el conjunto de datos completo. ¿Qué tan buena es realmente nuestra estimación? ¿Qué tan seguros estamos de que se generalizará para describir nuevos datos que aún no hemos visto?

Una solución a esto es simplemente recopilar más datos y verificar el MSE en este nuevo conjunto de datos con los parámetros estimados previamente. Sin embargo, esto no siempre es factible y aún deja abierta la cuestión de cuán cuantificablemente confiados estamos en la precisión de nuestro modelo.

En la Sección 1, exploraremos cómo implementar bootstrapping. En la Sección 2, crearemos intervalos de confianza de nuestras estimaciones utilizando el método de arranque.

---
# Configuraciones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#@title Configuraciones de Figuras
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [ ]:
#@title Funciones de ayuda
def solve_normal_eqn(x, y):
  """Solve the normal equations to produce the value of theta_hat that minimizes
    MSE.

    Args:
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.
    thata_hat (float): An estimate of the slope parameter.

  Returns:
    float: the value for theta_hat arrived from minimizing MSE
  """
  theta_hat = (x.T @ y) / (x.T @ x)
  return theta_hat

---
# Sección 1: Bootstrapping

Bootstrapping es un método ampliamente aplicable para evaluar la confianza/incertidumbre acerca de los parámetros estimados, originalmente fue propuesto por Bradley Efron. La idea es generar muchos conjuntos de datos sintéticos nuevos a partir del conjunto de datos verdadero inicial mediante un muestreo aleatorio de él, luego encontrar estimadores para cada uno de estos nuevos conjuntos de datos y, finalmente, observar la distribución de todos estos estimadores para cuantificar nuestra confianza.

Tenga en cuenta que cada nuevo conjunto de datos remuestreado tendrá el mismo tamaño que el original, con los nuevos puntos de datos muestreados con reemplazo, es decir, podemos repetir el mismo punto de datos varias veces. También tenga en cuenta que en la práctica necesitamos muchos conjuntos de datos remuestreados, aquí usamos 2000.

Para explorar esta idea, comenzaremos nuevamente con nuestras muestras ruidosas a lo largo de la línea $ y_n = 1.2x_n + \epsilon_n $, pero esta vez solo usaremos la mitad de los puntos de datos como la última vez (15 en lugar de 30).

In [ ]:
#@title

#@markdown Ejecuta esta celda para simular algo de data

# setting a fixed seed to our random number generator ensures we will always
# get the same psuedorandom number sequence
np.random.seed(121)

# Let's set some parameters
theta = 1.2
n_samples = 15

# Draw x and then calculate y
x = 10 * np.random.rand(n_samples)  # sample from a uniform distribution over [0,10)
noise = np.random.randn(n_samples)  # sample from a standard normal distribution
y = theta * x + noise

fig, ax = plt.subplots()
ax.scatter(x, y)  # produces a scatter plot
ax.set(xlabel='x', ylabel='y');

### Ejercicio 1: Volver a muestrear el conjunto de datos con reemplazo

En este ejercicio, implementará un método para volver a muestrear un conjunto de datos con reemplazo. El método acepta matrices $ x $ y $ y $. Debería devolver un nuevo conjunto de matrices $ x'$ y $ y' $ que se crean mediante un muestreo aleatorio de los originales.

Luego compararemos el conjunto de datos original con un conjunto de datos remuestreado.

TIP: La función [numpy.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) sería útil aquí.

In [ ]:
def resample_with_replacement(x, y):
  """Resample data points with replacement from the dataset of `x` inputs and
  `y` measurements.

  Args:
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.

  Returns:
    ndarray, ndarray: The newly resampled `x` and `y` data points.
  """

  # Obtenga una variedad de índices para puntos remuestreados
  sample_idx = ...

  # Muestra de x e y de acuerdo con sample_idx
  x_ = ...
  y_ = ...
  return x_, y_


fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))
ax1.scatter(x, y)
ax1.set(title='Original', xlabel='x', ylabel='y')

# Descomente a continuación para probar su función
#x_, y_ = resample_with_replacement(x, y)
#ax2.scatter(x_, y_, color='c')

ax2.set(title='Remuestreado', xlabel='x', ylabel='y',
        xlim=ax1.get_xlim(), ylim=ax1.get_ylim());

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial3_Solution_fe37e229.py)

*Salida de ejemplo:*

<img alt='Sugerencia de solución' align='left' width=845 height=341 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D3_ModelFitting/static/W1D3_Tutorial3_Solution_fe37e229_0.png>

En la gráfica remuestreada de la derecha, el número real de puntos es el mismo, pero algunos se han repetido, por lo que solo se muestran una vez.

Ahora que tenemos una forma de volver a muestrear los datos, podemos usarla en el proceso de arranque completo.

### Ejercicio 2: Estimaciones de Bootstrap

En este ejercicio, implementará un método para ejecutar el proceso de arranque de generar un conjunto de valores $ \hat\theta $ a partir de un conjunto de datos de entradas $ x $ y medidas $ y $. Debe usar `resample_with_replacement` aquí, y también puede invocar la función auxiliar `solve_normal_eqn` del Tutorial 1 para producir el estimador basado en MSE.

Luego usaremos esta función para ver theta_hat de diferentes muestras.

In [ ]:
def bootstrap_estimates(x, y, n=2000):
  """Generate a set of theta_hat estimates using the bootstrap method.

  Args:
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.
    n (int): The number of estimates to compute

  Returns:
    ndarray: An array of estimated parameters with size (n,)
  """
  theta_hats = np.zeros(n)

  # Bucle sobre el número de estimaciones
  for i in range(n):

    # Remuestrear x e y
    x_, y_ = ...

    # Calcule theta_hat para esta muestra
    theta_hats[i] = ...

  return theta_hats


np.random.seed(123)  # establecer semilla aleatoria para comprobar soluciones

# Descomente a continuación para probar la función
# theta_hats = bootstrap_estimates(x, y, n=2000)
# print(theta_hats[0:5])

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial3_Solution_315a8d02.py)



Deberías ver `[1.27550888 1.17317819 1.18198819 1.25329255 1.20714664]` como las cinco primeras estimaciones.

Ahora que tenemos nuestras estimaciones de arranque, podemos visualizar todos los modelos potenciales (modelos calculados con diferentes remuestreos) juntos para ver qué tan distribuidos están.

In [ ]:
#@title
#@markdown Ejecute esta celda para visualizar todos los modelos potenciales

fig, ax = plt.subplots()

# For each theta_hat, plot model
theta_hats = bootstrap_estimates(x, y, n=2000)
for i, theta_hat in enumerate(theta_hats):
  y_hat = theta_hat * x
  ax.plot(x, y_hat, c='r', alpha=0.01, label='Resampled Fits' if i==0 else '')

# Plot observed data
ax.scatter(x, y, label='Observed')

# Plot true fit data
y_true = theta * x
ax.plot(x, y_true, 'g', linewidth=2, label='True Model')

ax.set(
  title='Bootstrapped Slope Estimation',
  xlabel='x',
  ylabel='y'
)

# Change legend line alpha property
handles, labels = ax.get_legend_handles_labels()
handles[0].set_alpha(1)

ax.legend();

¡Esto se ve bastante bien! Las estimaciones de bootstrap se extienden alrededor del modelo real, como hubiéramos esperado. Tenga en cuenta que aquí tenemos el lujo de conocer el valor real básico de $ \ theta $, pero en las aplicaciones estamos tratando de adivinarlo a partir de los datos. Por lo tanto, evaluar la calidad de las estimaciones basadas en datos finitos es una tarea de fundamental importancia en el análisis de datos.

---
# Sección 2: Intervalos de confianza

Cuantifiquemos ahora qué tan incierta es nuestra pendiente estimada. Lo hacemos calculando intervalos de confianza (CIs) a partir de nuestras estimaciones de arranque. El enfoque más directo es calcular percentiles a partir de la distribución empírica de estimaciones bootstrap. Tenga en cuenta que esto es ampliamente aplicable ya que no estamos asumiendo que esta distribución empírica sea gaussiana.

In [ ]:
#@title

#@markdown Ejecute esta celda para trazar Intervalos de Confianza de arranque

theta_hats = bootstrap_estimates(x, y, n=2000)
print(f"mean = {np.mean(theta_hats):.2f}, std = {np.std(theta_hats):.2f}")

fig, ax = plt.subplots()
ax.hist(theta_hats, bins=20, facecolor='C1', alpha=0.75)
ax.axvline(theta, c='g', label=r'True $\theta$')
ax.axvline(np.percentile(theta_hats, 50), color='r', label='Median')
ax.axvline(np.percentile(theta_hats, 2.5), color='b', label='95% CI')
ax.axvline(np.percentile(theta_hats, 97.5), color='b')
ax.legend()
ax.set(
    title='Bootstrapped Confidence Interval',
    xlabel=r'$\hat{{\theta}}$',
    ylabel='count',
    xlim=[1.0, 1.5]
);

Al observar la distribución de los valores de $ \hat{\theta} $ bootstrap, vemos que el verdadero $ \theta $ cae dentro del intervalo de confianza del 95%, lo que es reasegurador. También vemos que el valor $ \theta = 1 $ no cae dentro del intervalo de confianza. A partir de esto, rechazaríamos la hipótesis de que la pendiente fuera 1.

---
# Resumen

- Bootstrapping es un procedimiento de remuestreo que permite construir intervalos de confianza alrededor de valores de parámetros inferidos
- es un método ampliamente aplicable y muy práctico que se basa en el poder computacional y generadores de números pseudoaleatorios (a diferencia de los enfoques más clásicos que dependen de derivaciones analíticas)